In [2]:
# import logging

# logging.basicConfig(level=logging.DEBUG)


from src.serial_utilities.serial_utilities import (
    detect_baudrate,
    detect_port,
    test_connection,
    list_ports_info,
    print_ports_info,
    scan_ports,
    TestCase,
)

print_ports_info()
test_connection("COM3", 115200, test_case=TestCase.default())
scan_ports(test_case=TestCase.default())


Serial connection error: could not open port 'COM3': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)


-----------------------------------------------------------------------------------------------------------------------------
Port       Description                    VID   PID   Manufacturer              Serial Number             Location                 
-----------------------------------------------------------------------------------------------------------------------------
COM4       USB-SERIAL CH340 (COM4)        6790  29987 wch.cn                    N/A                       1-4                      
-----------------------------------------------------------------------------------------------------------------------------


{'COM4': 115200}

In [3]:
from src.stepper.stepper_move import Enable, Disable, Jog, Move, EStop, SyncMove
from src.stepper.stepper_home import SetHome, Home, StopHome, RetrieveHomeParam, SetHomeParam, GetHomeStatus
from src.stepper.stepper_get import (
    GetBusVoltage,
    GetPhaseCurrent,
    GetEncoderValue,
    GetVersion,
    GetMotorRH,
    GetPID,
    GetConfig,
    GetStatus,
    GetError,
    GetSysStatus,
)
from src.stepper.stepper_set import (
    SetMicrostep,
    SetID,
    SetLoopMode,
    SetOpenLoopCurrent,
    SetPID,
    SetStartSpeed,
    SetReduction,
    Microstep,
    LoopMode,
    OpenLoopCurrent,
    Kpid,
    SpeedReduction,
    StartSpeedParams,
    SetConfig,
)

from src.stepper.stepper_parameters import PositionParams, JogParams, DeviceParams
from src.stepper.stepper_constants import Address, SyncFlag, Direction, Speed, Acceleration, PulseCount, AbsoluteFlag, HomingMode

from serial import Serial
 
# from time import sleep
# import logging
# logging.basicConfig(level=logging.DEBUG)


serial_connection = Serial("COM4", 115200, timeout=0.1)

device_params = DeviceParams(
    serial_connection=serial_connection,
    address=Address(0x01),
)

position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(500),
    acceleration=Acceleration(127),
    pulse_count=PulseCount(160),
    absolute=AbsoluteFlag.RELATIVE,
)

jog_params = JogParams(
    direction=Direction.CW,
    speed=Speed(1),
    acceleration=Acceleration(255),
)

In [6]:
Enable(device=device_params).status
Move(device=device_params, params=position_params).status


'SUCCESS'

In [5]:
Disable(device=device_params).status
Move(device=device_params, params=position_params).status

'CONDITIONAL_ERROR'

In [7]:
Jog(device=device_params, params=jog_params).status

'SUCCESS'

In [8]:
EStop(device=device_params).status

'SUCCESS'

In [9]:
Move(device=device_params, params=position_params, setting=SyncFlag.SYNC).is_success
SyncMove(device=device_params).status

'SUCCESS'

In [10]:
position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(100),
    acceleration=Acceleration(255),
    pulse_count=PulseCount(6400),
    absolute=AbsoluteFlag.RELATIVE,
)

SetHome(device=device_params).status
Move(device=device_params, params=position_params).status



'SUCCESS'

In [11]:
Home(device=device_params, params=HomingMode.MULTI_TURN_UNLIMITED).status
# sleep(1)
# StopHome(device=device_params).status

'SUCCESS'

In [12]:
RetrieveHomeParam(device=device_params).data

{'homing_mode': 'SINGLE_TURN_NEAREST',
 'homing_direction': 'CW',
 'homing_speed (RPM)': 30.0,
 'homing_timeout': 10000,
 'collision_detection_speed (RPM)': 300.0,
 'collision_detection_current (mA)': 800.0,
 'collision_detection_time (ms)': 100.0,
 'auto_home': 'DISABLE'}

In [13]:
homing_params = RetrieveHomeParam(device=device_params).raw_data


In [14]:
homing_params.bytes

b"\x00\x00\x00\x1e\x00\x00'\x10\x01,\x03 \x00d\x00"

In [15]:
SetHomeParam(device=device_params, params=homing_params).status

'SUCCESS'

In [16]:
Move(device=device_params, params=position_params).status



'CONDITIONAL_ERROR'

In [17]:
Home(device=device_params, params=HomingMode.SINGLE_TURN_NEAREST).status

'CONDITIONAL_ERROR'

In [18]:
GetHomeStatus(device=device_params).raw_data.data_dict

{'encoder_ready': True,
 'encoder_calibrated': True,
 'homing_active': True,
 'homing_failed': False}

In [27]:
from src.stepper.stepper_sys import CalibrateEncoder

calibrate_encoder = CalibrateEncoder
calibrate_encoder.unlock()
calibrate_encoder(device=device_params).status


'CONDITIONAL_ERROR'

In [24]:

print(GetVersion(device=device_params).data)
print(GetMotorRH(device=device_params).data)
print(GetPID(device=device_params).data)
print(GetConfig(device=device_params).data)
print(GetStatus(device=device_params).data)
print(GetError(device=device_params).data)
print(GetSysStatus(device=device_params).data)
print(GetBusVoltage(device=device_params).data)
print(GetPhaseCurrent(device=device_params).data)
print(GetEncoderValue(device=device_params).data)



{'firmware_version': 255, 'hardware_version': 120}
{'phase_resistance (mOhm)': 3486.0, 'phase_inductance (uH)': 1170.0}
{'pid_p': 32000, 'pid_i': 10, 'pid_d': 32000}
{'stepper_type': 'D18', 'control_mode': 'PUL_FOC', 'communication_mode': 'UART', 'enable_level': 'HOLD', 'default_direction': 'CW', 'microsteps': 16, 'microstep_interp': 'ENABLE', 'screen_off': 'DISABLE', 'open_loop_current (mA)': 800.0, 'max_closed_loop_current (mA)': 3000.0, 'max_voltage (mV)': 5000.0, 'baud_rate': 'BAUD_115200', 'can_rate': 'CAN_500K', 'address': 1, 'checksum_mode': 'FIXED', 'response_mode': 'RECEIVE', 'stall_protect': 'ENABLE', 'stall_speed (RPM)': 8.0, 'stall_current (mA)': 2600.0, 'stall_time (ms)': 2000.0, 'on_target_window': 0.3}
{'enabled': True, 'in_position': True, 'stalled': False, 'stall_protection_active': False}
{'position_error (deg)': -0.1263427734375}
{'bus_voltage (mV)': 11934.0, 'bus_phase_current (mA)': 31.0, 'calibrated_encoder_value (deg)': 11.5301513671875, 'stepper_target_position 

In [37]:
from itertools import chain

sys_status = GetSysStatus(device=device_params).raw_data
config_params = GetConfig(device=device_params).raw_data
for key, value in sorted(chain(sys_status.data_dict.items(), config_params.data_dict.items())):
    print(f"{key}: {value}")


address: 1
baud_rate: BAUD_115200
bus_phase_current (mA): 46.0
bus_voltage (mV): 11956.0
calibrated_encoder_value (deg): 359.8956298828125
can_rate: CAN_500K
checksum_mode: FIXED
communication_mode: UART
control_mode: PUL_FOC
default_direction: CW
enable_level: HOLD
encoder_calibrated: True
encoder_ready: True
homing_failed: False
is_homing: True
max_closed_loop_current (mA): 3000.0
max_voltage (mV): 5000.0
microstep_interp: ENABLE
microsteps: 16
on_target_window: 0.3
open_loop_current (mA): 800.0
response_mode: RECEIVE
screen_off: DISABLE
stall_current (mA): 2600.0
stall_protect: ENABLE
stall_speed (RPM): 8.0
stall_time (ms): 2000.0
stepper_enabled: True
stepper_in_position: True
stepper_position_error (deg): 0.1043701171875
stepper_real_time_position (deg): -0.1043701171875
stepper_real_time_speed (RPM): 0.0
stepper_stall_protection_active: False
stepper_stalled: False
stepper_target_position (deg): 0.0
stepper_type: D18


In [38]:
Set_Microstep = SetMicrostep
Set_Microstep.unlock()
Set_Microstep(device=device_params, params=Microstep(value=16)).status

'SUCCESS'

In [39]:
Set_ID = SetID
Set_ID.unlock()
Set_ID(device=device_params, params=Address(value=1)).status


'SUCCESS'

In [40]:
SetLoopMode = SetLoopMode
SetLoopMode.unlock()
SetLoopMode(device=device_params, params=LoopMode.CLOSED).status


'SUCCESS'

In [41]:
Set_open_loop_current = SetOpenLoopCurrent
Set_open_loop_current.unlock()
Set_open_loop_current(device=device_params, params=OpenLoopCurrent(value=800)).status

'SUCCESS'

In [42]:
PID_params = GetPID(device=device_params).raw_data
print(PID_params.data_dict)
Set_PID = SetPID
Set_PID.unlock()
Set_PID(device=device_params, params=PID_params).status


{'pid_p': 32000, 'pid_i': 10, 'pid_d': 32000}


'SUCCESS'

In [43]:
from src.stepper.stepper_constants import EnablePin
start_speed_params = StartSpeedParams(
    direction=Direction.CW,
    speed=Speed(1000),
    acceleration=Acceleration(255),
    en_control=EnablePin.ENABLE,
)
Set_StartSpeed = SetStartSpeed
Set_StartSpeed.unlock()
Set_StartSpeed(device=device_params, params=start_speed_params).status

'SUCCESS'

In [50]:
from src.stepper.stepper_constants import StallTime

config_params = GetConfig(device=device_params).raw_data
print("Initial Config:")
for key, value in sorted(config_params.data_dict.items()):
    print(f"{key}: {value}")

config_params.microstep = Microstep(value=32)
config_params.stall_time = StallTime(value=100)

Set_Config = SetConfig
Set_Config.unlock()
Set_Config(device=device_params, params=config_params).status

config_params = GetConfig(device=device_params).raw_data
print("-----------------------")
print("Changed Config:")
for key, value in sorted(config_params.data_dict.items()):
    print(f"{key}: {value}")

config_params.microstep = Microstep(Microstep.default)
config_params.stall_time = StallTime(StallTime.default)

Set_Config = SetConfig
Set_Config.unlock()
Set_Config(device=device_params, params=config_params).status


config_params = GetConfig(device=device_params).raw_data
print("-----------------------")
print("Changed Config:")
for key, value in sorted(config_params.data_dict.items()):
    print(f"{key}: {value}")



Initial Config:
address: 1
baud_rate: BAUD_115200
can_rate: CAN_500K
checksum_mode: FIXED
communication_mode: UART
control_mode: PUL_FOC
default_direction: CW
enable_level: HOLD
max_closed_loop_current (mA): 3000.0
max_voltage (mV): 5000.0
microstep_interp: ENABLE
microsteps: 16
on_target_window: 0.3
open_loop_current (mA): 800.0
response_mode: RECEIVE
screen_off: DISABLE
stall_current (mA): 2600.0
stall_protect: ENABLE
stall_speed (RPM): 8.0
stall_time (ms): 4000.0
stepper_type: D18
-----------------------
Changed Config:
address: 1
baud_rate: BAUD_115200
can_rate: CAN_500K
checksum_mode: FIXED
communication_mode: UART
control_mode: PUL_FOC
default_direction: CW
enable_level: HOLD
max_closed_loop_current (mA): 3000.0
max_voltage (mV): 5000.0
microstep_interp: ENABLE
microsteps: 16
on_target_window: 0.3
open_loop_current (mA): 800.0
response_mode: RECEIVE
screen_off: DISABLE
stall_current (mA): 2600.0
stall_protect: ENABLE
stall_speed (RPM): 8.0
stall_time (ms): 100.0
stepper_type: D18